<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [60]:
import pandas as pd
import psycopg2


In [61]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [64]:
query_vac_count = '''
                  SELECT COUNT(1)
                  FROM vacancies
                  '''

In [65]:
vac_count = pd.read_sql_query(query_vac_count, connection)
vac_count

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [66]:
query_empl_count = '''
                   SELECT COUNT(1)
                   FROM employers
                   '''

In [67]:
empl_count = pd.read_sql_query(query_empl_count, connection)
empl_count

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [68]:
query_areas_count = '''
                    SELECT COUNT(1)
                    FROM areas
                    '''

In [69]:
areas_count = pd.read_sql_query(query_areas_count, connection)
areas_count

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [70]:
query_idustries_count = '''
                        SELECT COUNT(1)
                        FROM industries
                        '''

In [71]:
idustries_count = pd.read_sql_query(query_idustries_count, connection)
idustries_count

,count
0,294



##### Предварительный анализ данных показывает:
* Общее количество вакансий приближается к 50 тысячам.
* В базе зарегестрированы 29,5 тысяч работодателей.
* База содержит 1362 различных региона.
* Указаны 294 сферы деятельности.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [72]:
query_cnt_area = '''
                 SELECT
                   COUNT(1) AS cnt,
                   a.name AS area
                 FROM vacancies AS v
                 JOIN areas AS a
                   ON v.area_id = a.id
                 GROUP BY a.name
                 ORDER BY cnt DESC
                 LIMIT 5
                 '''

In [73]:
cnt_area = pd.read_sql_query(query_cnt_area, connection)
cnt_area

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [74]:
query_salary_on_count = '''
                        SELECT
                          COUNT(1)
                        FROM vacancies 
                        WHERE (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
                        '''

In [75]:
salary_on_count = pd.read_sql_query(query_salary_on_count, connection)
salary_on_count

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [76]:
query_salary_avg = '''
                   SELECT
                     AVG(salary_from)::INT AS avg_from,
                     AVG(salary_to)::INT AS avg_to
                   FROM vacancies 
                   '''

In [77]:
alary_avg = pd.read_sql_query(query_salary_avg, connection)
alary_avg

,avg_from,avg_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [78]:
query_vacancies_types = '''
                        SELECT
                             schedule,
                             employment,
                             COUNT(1)
                        FROM vacancies 
                        GROUP BY schedule, employment
                        ORDER BY 3 DESC
                        LIMIT 5
                        '''

In [79]:
vacancies_types = pd.read_sql_query(query_vacancies_types, connection)
vacancies_types

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [80]:
query_experience = '''
                   SELECT
                      experience,
                      COUNT(1)
                   FROM vacancies 
                   GROUP BY experience
                   ORDER BY 2
                   '''

In [81]:
experience = pd.read_sql_query(query_experience, connection)
experience

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


##### Детальный анализ вакансий позволяет отметить:
* Наибольшее количество вакансий в Москве. Также в лидерах по числу вакансий такие города как Санк-Петербург, Минск, Новосибирск, Алматы.
* Примерно в половине вакансий указывается предполагаемый размер зарплаты.
* Среднее значение указываемое для нижней и верхней зарплатной вилки соответсвует 71 тысяче рублей и 110 тысяч рублей.
* Наибольшее количество вакансий требуют 'полный день' и 'полную занятость'. Также довольно распротранена 'полная занятость' в сочетании с 'удаленной работой'.
* Чаще всего требуемый опыт работы от 1 года до 3 лет. Редко запращивается опыт работы более 6 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [82]:
query_employers_vac_count = '''
                            SELECT
                                e.name,
                                COUNT(1)
                            FROM vacancies AS v
                              JOIN employers AS e
                                ON v.employer_id = e.id
                            GROUP BY e.name
                            ORDER BY 2 DESC
                            LIMIT 5
                            '''

In [83]:
employers_vac_count = pd.read_sql_query(query_employers_vac_count, connection)
employers_vac_count

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [84]:
query_area_empl_vac = '''
                      WITH area_empl AS
                        (
                        SELECT 
                            a.id,
                            a.name,
                        COUNT(e.id) AS num_emloyers
                        FROM areas AS a
                        LEFT JOIN employers AS e
                          ON a.id = e.area
                        GROUP BY a.id, a.name
                        )

                      SELECT
                          a.name AS area,
                          num_emloyers,
                      COUNT(v.id) AS num_vacancies
                      FROM area_empl AS a
                        LEFT JOIN vacancies AS v
                          ON a.id = v.area_id
                      GROUP BY a.name, a.num_emloyers
                      HAVING COUNT(v.id) = 0
                      ORDER BY num_emloyers DESC
                      LIMIT 5
                      '''

In [85]:
area_empl_vac = pd.read_sql_query(query_area_empl_vac, connection)
area_empl_vac

,area,num_emloyers,num_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [86]:
query_empl_vac_areas = '''
                       SELECT
                           e.name,
                           COUNT(DISTINCT v.area_id) AS vac_areas
                       FROM employers AS e
                         JOIN vacancies AS v
                           ON e.id = v.employer_id
                       GROUP BY e.name
                       ORDER BY vac_areas DESC
                       LIMIT 5
                       '''

In [87]:
empl_vac_areas = pd.read_sql_query(query_empl_vac_areas, connection)
empl_vac_areas

,name,vac_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [88]:
query_employers_industries_off = '''
                                 SELECT
                                   COUNT(1)
                                 FROM employers AS e
                                   LEFT JOIN employers_industries AS i
                                     ON e.id = i.employer_id
                                 WHERE i.industry_id IS NULL
                                 '''

In [89]:
employers_industries_off = pd.read_sql_query(query_employers_industries_off, connection)
employers_industries_off

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [90]:
query_employer_3_with_4_industries = '''
                                     SELECT
                                       e.name,
                                       COUNT(DISTINCT i.industry_id) AS industries_count
                                     FROM employers AS e
                                       LEFT JOIN employers_industries AS i
                                         ON e.id = i.employer_id
                                     GROUP BY e.name
                                     HAVING COUNT(DISTINCT i.industry_id) = 4
                                     ORDER BY e.name
                                     OFFSET 2
                                     LIMIT 1
                                     '''

In [91]:
employer_3_with_4_industries = pd.read_sql_query(query_employer_3_with_4_industries, connection)
employer_3_with_4_industries

,name,industries_count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [92]:
query_employers_devs = '''
                       SELECT
                         COUNT(1)
                       FROM employers_industries AS e
                         JOIN industries AS i
                           ON e.industry_id = i.id
                       WHERE i.name = 'Разработка программного обеспечения'
                       '''

In [93]:
employers_devs = pd.read_sql_query(query_employers_devs, connection)
employers_devs

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [94]:
import requests
from bs4 import BeautifulSoup
URL = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [95]:
table_cities = soup.find('table', class_='standard sortable')  # находим таблицу с городами-миллиониками
cities_soup = table_cities.find_all('a', title=True)  # находим строки с наименованиями городов по тегу <a> и аттрибуту title

list_cities = [a['title'] for a in cities_soup]  # создаем список загловоков из таблицы википедии, который включает города-миллионики
display(list_cities)

['Всероссийская перепись населения (2020—2021)',
 'Всероссийская перепись населения (2010)',
 'Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [96]:
areas_millions_cities = tuple(list_cities[2:]) # создаем tuple городов-миллионников, обрезая пару ненужных заголовков из списка

In [97]:
query_yandex_vac = f'''
                   WITH yandex_cities AS 
                                     (
                                      SELECT
                                        a.name AS cities,
                                        COUNT(1) AS cnt
                                      FROM vacancies AS v
                                        JOIN employers AS e
                                          ON v.employer_id = e.id
                                        JOIN areas as a
                                          ON v.area_id = a.id
                                      WHERE e.name = 'Яндекс' AND a.name IN {areas_millions_cities}
                                      GROUP BY a.name
                                      )
                   SELECT cities, cnt
                   FROM yandex_cities
                   UNION ALL
                   SELECT 'Total', SUM(cnt)::INT
                   FROM yandex_cities
                   ORDER BY cnt
                   '''

In [98]:
yandex_vac = pd.read_sql_query(query_yandex_vac, connection)
yandex_vac

,cities,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


##### При анализе работодателей можно отметить:

* Явным лидиром по числу вакансий в базе явялется компания Яндекс. Другие компании с наибольшим количеством ваканисий: Ростелеком, Тинькофф, Сбер.
* Классификация регионов включает в себя такие регионы работодателей как, например, Россия и Казахстан. В тоже время такие определения регионов не используются при составлении вакансий.
* Для наибольшего количества регионов предоставляют вакансии Яндекс, Ростелеком, Спецремонт.
* Более 3500 компаний укзали сферой деятельности разработку программного обеспечения.
* Лидер по общему числу вакансий - компания Яндекс. Представлена во всех городах-миллиониках России.

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [99]:
query_vacancies_data = '''           
                       SELECT
                       COUNT(1) AS data_vac_cnt
                       FROM vacancies AS v
                       WHERE LOWER(v.name) LIKE '%data%' OR LOWER(v.name) LIKE '%данн%'
                       '''

In [100]:
vacancies_data = pd.read_sql_query(query_vacancies_data, connection)
vacancies_data

,data_vac_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [121]:
data_key_words = ['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%']

query_vacancies_data = f'''           
                        SELECT
                        COUNT(1) AS data_vac_cnt
                        FROM vacancies AS v
                        WHERE (
                               LOWER(v.name) LIKE ANY (array{data_key_words})
                                   OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                              )
                              AND (
                                   LOWER(v.name) LIKE '%junior%' 
                                   OR v.experience = 'Нет опыта' 
                                   OR v.employment = 'Стажировка'
                                   )
                        '''

In [122]:
vacancies_data = pd.read_sql_query(query_vacancies_data, connection)
vacancies_data

,data_vac_cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [103]:
# создаем фильтр для вакансий связанных с DS
vacancies_ds_where_filter = '''
                            (
                            LOWER(v.name) LIKE ANY (array['%data scientist%','%data science%',
                                                          '%исследователь данных%','%machine learning%',
                                                          '%машинн%обучен%'])
                            OR 
                            (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                            )
                            '''

In [104]:
query_vac_data_sql = f'''           
                      SELECT
                      COUNT(1)
                      FROM vacancies AS v
                      WHERE {vacancies_ds_where_filter}
                            AND 
                            (v.key_skills LIKE '%SQL%' OR v.key_skills LIKE '%postgres%') 
                      '''

In [105]:
vac_data_sql = pd.read_sql_query(query_vac_data_sql, connection)
vac_data_sql

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [106]:
query_vac_data_python = f'''           
                        SELECT
                        COUNT(1)
                        FROM vacancies AS v
                        WHERE {vacancies_ds_where_filter}
                              AND LOWER(v.key_skills) LIKE '%python%'          
                       '''

In [107]:
vac_data_python = pd.read_sql_query(query_vac_data_python, connection)
vac_data_python

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [108]:
query_vac_data_skills = f'''           
                        SELECT AVG(LENGTH(v.key_skills) - LENGTH(replace(v.key_skills, $$\t$$,'')) + 1)::DEC(18,2) AS avg_skills_cnt
                        FROM vacancies AS v
                        WHERE {vacancies_ds_where_filter}
                              AND v.key_skills IS NOT NULL  
                        '''

In [109]:
vac_data_skills = pd.read_sql_query(query_vac_data_skills, connection)
vac_data_skills

,avg_skills_cnt
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [110]:
query_vac_data_salary = f'''           
                         SELECT v.experience,
                                AVG((COALESCE(((v.salary_from + v.salary_to)/2), v.salary_from, v.salary_to)))::INT AS avg_salary
                         FROM vacancies AS v
                         WHERE {vacancies_ds_where_filter}
                               AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                         GROUP BY v.experience
                         '''

In [111]:
vac_data_salary = pd.read_sql_query(query_vac_data_salary, connection)
vac_data_salary

,experience,avg_salary
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


##### Выводы по предметному анализу:
* База содержит 1771 вакансию относящуюся к сфере работы с данными.
* Начинающих дата саентистов ожидает 51 предложение.
* 201 ваканcия предполагают работу c SQL.
* Из 1771 относящимся к данным, отдельно упоминается в качестве ключевого навыка Python в 351 вакансии.
* Ожидаемая средняя зарплата в сфере данных значительно зависит от предполагаемого опыта работы. Стартуя от 74 тысяч рублей для соискателей без опыта, и доходит до 243 тысяч рублей при опыте от 3 до 6 лет.
* Вакансии относящиеся к работе с данными предполагают большой набор запрашиваемых требований. Среднее количество запрашиваемых ключевых навыков больше 6.

***

# 7. Дополнительный анализ

7.1. Проанализируем вакансии DS в разрезе требуемого графика работы и условия занятости.

In [112]:
query_ds_schedule_salary = f'''
                           SELECT schedule,
                                  employment,
                                  COUNT(1) as cnt_vac
                           FROM vacancies AS v
                           WHERE {vacancies_ds_where_filter}
                           GROUP BY schedule, employment
                           ORDER BY cnt_vac DESC
                           '''
ds_schedule_salary = pd.read_sql_query(query_ds_schedule_salary, connection)
ds_schedule_salary

,schedule,employment,cnt_vac
0,Полный день,Полная занятость,319
1,Удаленная работа,Полная занятость,110
2,Гибкий график,Полная занятость,37
3,Гибкий график,Частичная занятость,4
4,Удаленная работа,Частичная занятость,3
5,Полный день,Стажировка,2
6,Удаленная работа,Проектная работа,2
7,Гибкий график,Стажировка,2
8,Полный день,Частичная занятость,1


* Наибольшее количество вакансий в DS предполагает полный день и полную занятоость.
* Удаленная работа с полной занятостью тоже довольно популярный вариант.

7.2. Какие компании лидируют по количеству вакансий в сфере DS.

In [123]:
query_ds_employers = f'''
                     SELECT 
                            e.name,
                            COUNT(1)
                     FROM vacancies AS v
                     LEFT JOIN employers AS e
                            ON v.employer_id = e.id
                     WHERE {vacancies_ds_where_filter}      
                     GROUP BY e.name
                     ORDER BY 2 DESC
                     LIMIT 5
                     '''
ds_employers = pd.read_sql_query(query_ds_employers, connection)
ds_employers

,name,count
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11


* В сфере DS наибольшее число компаний у СБЕРА. Также значитальное количество вакансий представлено компанией Bell Integrator и Банком ВТБ.

In [114]:
# закрываем соединение после окончания работы
connection.close()

# Общий вывод по проекту

* База вакансий HeadHunter содержит почти 50 тысяч вакансий.
* Георафия регионов предствалена как городами России, так и городами ближайший зарубежных стран. 
* В базе зарегестрировано 29 тысяч работодателей.
* Крупнейшие работадатели: Яндекс, Ростелеком, Тинькофф, Сбер.
* В базе более 1700 вакансий связанных с data scene.
* Ожидаемая заплата в сфере DS сильно зависит от опыта работы. 
* Cоставялет в среднем 74 тысяч рублей для соискателей без опыта, возростая до 243 тысяч рублей при опыте от 3 до 6 лет.
* Ваканисии DS в базе чаще оринтированы на полную занятость и полный день. Также предлагаются удаленные вариант работы.
* Наибольшее количество вакансий DS у Сбера, Bell Integrator и Банк ВТБ.